In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
# from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from template_matching import enhance, find_matches_and_homography, adjust_keypoints
import random
import pickle
import cv2
from copy import copy
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool, Manager
import pyarrow.parquet as pq

import matplotlib.cm as cm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

<h1> Extract base data from database </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS'])))
query = """
    select * from research.fish_metadata a left join keypoint_annotations b
    on a.left_url = b.left_image_url 
    where b.keypoints is not null and b.is_qa = false;
"""
bdf = rds_access_utils.extract_from_database(query)

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS'])))
query = """
    select * from research.fish_metadata a left join keypoint_annotations b
    on a.left_url = b.left_image_url 
    where b.keypoints is not null and b.is_qa = false
    limit 1;
"""
bdf = rds_access_utils.extract_from_database(query)

<h1> Aggregate Parquet Data and Merge with Base Data </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
gen = s3_access_utils.get_matching_s3_keys('aquabyte-research', prefix='template-matching/2019-10-23T20', suffixes=['.parquet'])
keys = []
for key in gen:
    keys.append(key)

f = s3_access_utils.download_from_s3('aquabyte-research', keys[0])
pdf = pd.read_parquet(f)


In [ ]:
df = pd.merge(bdf, pdf[['left_image_url', 'plane']], how='inner', on='left_image_url')

<h1> Get adjusted world keypoints </h1>

In [ ]:
def get_adjusted_wkps(df):
    adjusted_wkps_list = []
    for idx, row in df.iterrows():
        keypoints = row.keypoints
        left_crop_metadata = row.left_crop_metadata
        right_crop_metadata = row.right_crop_metadata
        cm = row.camera_metadata
        
        if 'leftCrop' in keypoints and 'rightCrop' in keypoints:
            left_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['leftCrop']}
            right_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['rightCrop']}

            # crop the data
            l_width = left_crop_metadata['width']
            l_height = left_crop_metadata['height']
            r_width = right_crop_metadata['width']
            r_height = right_crop_metadata['height']
            padding = 100
            cropL_x_left = max(min([kp[0] for kp in left_keypoints_dict.values()]) - padding, 0)
            cropL_x_right = min(max([kp[0] for kp in left_keypoints_dict.values()]) + padding, l_width)
            cropL_y_top = max(min([kp[1] for kp in left_keypoints_dict.values()]) - padding, 0)
            cropL_y_bottom = min(max([kp[1] for kp in left_keypoints_dict.values()]) + padding, l_height)

            cropR_x_left = max(min([kp[0] for kp in right_keypoints_dict.values()]) - padding, 0)
            cropR_x_right = min(max([kp[0] for kp in right_keypoints_dict.values()]) + padding, r_width)
            cropR_y_top = max(min([kp[1] for kp in right_keypoints_dict.values()]) - padding, 0)
            cropR_y_bottom = min(max([kp[1] for kp in right_keypoints_dict.values()]) + padding, r_height)
            
            #modify keypoints
            modified_keypoints = {'leftCrop': [], 'rightCrop': []}
            for item in keypoints['leftCrop']:
                modified_item = copy(item)
                modified_item['xCrop'] = item['xCrop'] - cropL_x_left
                modified_item['yCrop'] = item['yCrop'] - cropL_y_top
                modified_keypoints['leftCrop'].append(modified_item)

            for item in keypoints['rightCrop']:
                modified_item = copy(item)
                modified_item['xCrop'] = item['xCrop'] - cropR_x_left
                modified_item['yCrop'] = item['yCrop'] - cropR_y_top
                modified_keypoints['rightCrop'].append(modified_item)
                
#             modified_keypoints['leftCrop']['BODY'] = {
#                 'keypointType': 'BODY',
#                 'x'
#             }
                
            H = np.array([list(x) for x in row.plane])
            adjusted_keypoints = adjust_keypoints(modified_keypoints, H)
            adjusted_wkps = pixel2world(adjusted_keypoints['leftCrop'], 
                                    adjusted_keypoints['rightCrop'],
                                    cm)
            
            adjusted_wkps_list.append(adjusted_wkps)
        else:
            adjusted_wkps_list.append(None)
    df['world_keypoints'] = adjusted_wkps_list
        



In [ ]:
get_adjusted_wkps(df)

In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.keypoints and 'rightCrop' in row.keypoints:
        return pixel2world(row.keypoints['leftCrop'], row.keypoints['rightCrop'], row.camera_metadata)
    else:
        return None
    
df['original_world_keypoints'] = df.apply(
    lambda x: get_world_keypoints(x), axis=1
)

In [ ]:
features_df['original_world_keypoints'] = df['original_world_keypoints']

In [ ]:
def plot_world_keypoints_3D(wkps, original_wkps):
    norm_wkps = _normalize_world_keypoints(wkps)
    body_parts = [k for k in norm_wkps.keys()]
    xs = [norm_wkps[bp][0] for bp in body_parts]
    ys = [norm_wkps[bp][1] for bp in body_parts]
    zs = [norm_wkps[bp][2] for bp in body_parts]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim3d(0, max(xs))
    ax.set_ylim3d(-0.3, 0.3)
    ax.set_zlim3d(-0.3, 0.3)
    ax.scatter(xs, ys, zs, color='red')

    norm_wkps = _normalize_world_keypoints(original_wkps)
    xs = [norm_wkps[bp][0] for bp in body_parts]
    ys = [norm_wkps[bp][1] for bp in body_parts]
    zs = [norm_wkps[bp][2] for bp in body_parts]

    ax.scatter(xs, ys, zs, color='blue')
    

%matplotlib notebook

idx = 1
cm = df.camera_metadata.iloc[idx]
wkps = df.world_keypoints.iloc[idx]
original_wkps = df.original_world_keypoints.iloc[idx]
plot_world_keypoints_3D(wkps, original_wkps)


<h1> Get the features dataframe from the base data with all pairwise distances </h1>

In [ ]:
features_data = defaultdict(list)

body_parts = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE',
    'UPPER_PRECAUDAL_PIT', 
    'LOWER_PRECAUDAL_PIT',
    'HYPURAL_PLATE'
])

body_parts_subset = sorted([
    'HYPURAL_PLATE',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
])


for idx, row in df.iterrows():
    world_keypoints = row.world_keypoints
    if world_keypoints:
        for i in range(len(body_parts)-1):
            for j in range(i+1, len(body_parts)):
                if body_parts[i] in body_parts_subset and body_parts[j] in body_parts_subset:
                    d = euclidean_distance(world_keypoints[body_parts[i]], 
                                           world_keypoints[body_parts[j]])
                    features_data['{}-{}'.format(i, j)].append(d)

            
        features_data['world_keypoints'].append(row.world_keypoints)
        features_data['weight'].append(row.weight)
        features_data['captured_at'].append(row.captured_at)
        features_data['gtsf_fish_identifier'].append(row.fish_id)
        features_data['pen_id'].append(row.pen_id)
        features_data['keypoint_annotation_id'].append(row.id)
        features_data['kf'].append(1e5 * row.weight / row['data']['lengthMms']**3) 
        features_data['length'].append(row['data']['lengthMms'] * 1e-3)
        features_data['width'].append(row['data']['widthMms'] * 1e-3 if 'widthMms' in row['data'] else None)
#         features_data['breadth'].append(row['data']['breadthMms'] * 1e-3 if 'breadthMms' in row['data'] else None)

features_df = pd.DataFrame(features_data)

# get rid of bad keypoint annotation ids

blacklisted_keypoint_annotation_ids = [
    606484, 
    635806, 
    637801, 
    508773, 
    640493, 
    639409, 
    648536, 
    507003,
    706002,
    507000,
    709298,
    714073,
    719239
]

# blacklist_mask = features_df['8-9'] > 1.0
blacklist_mask = features_df['2-6'] > 1.0
for kp_id in blacklisted_keypoint_annotation_ids:
    if blacklist_mask is None:
        blacklist_mask = features_df.keypoint_annotation_id == kp_id
    else:
        blacklist_mask = blacklist_mask | (features_df.keypoint_annotation_id == kp_id)
features_df = features_df[~blacklist_mask]



In [ ]:
# define all features

body_part_indices = [body_parts.index(bp) for bp in body_parts_subset]

pairwise_distance_columns = ['{0}-{1}'.format(x, y) for x, y in list(combinations(body_part_indices, 2))]
interaction_columns_quadratic = []
interaction_columns_cubic = []
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        col1 = pairwise_distance_columns[i]
        col2 = pairwise_distance_columns[j]
        interaction_column = '{},{}'.format(col1, col2)
        features_df[interaction_column] = features_df[col1] * features_df[col2]
        interaction_columns_quadratic.append(interaction_column)
        
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        for k in range(j, len(pairwise_distance_columns)):
            col1 = pairwise_distance_columns[i]
            col2 = pairwise_distance_columns[j]
            col3 = pairwise_distance_columns[k]
            interaction_column = '{},{},{}'.format(col1, col2, col3)
            features_df[interaction_column] = features_df[col1] * features_df[col2] * features_df[col3]
            interaction_columns_cubic.append(interaction_column)
            

<h1> Weight each datapoint based on the number of stereo images captured for that fish </h1>

<h1> Compute best fit plane </h1>

<h1> Compute and visualize SIFT features </h1>

In [ ]:
# SIFT based correction - functions

def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image 




def draw_matches(img1, kp1, img2, kp2, matches, matchesMask, color=None, drawFeatures=True): 
    if len(img1.shape) == 3:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], img1.shape[2])
    elif len(img1.shape) == 2:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1])
    new_img = np.zeros(new_shape, type(img1.flat[0]))  
    # Place images onto the new image.
    new_img[0:img1.shape[0],0:img1.shape[1]] = img1
    new_img[0:img2.shape[0],img1.shape[1]:img1.shape[1]+img2.shape[1]] = img2
    
    if drawFeatures==False:
        return new_img

    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    r = 15
    thickness = 3
    if color:
        c = color
    i=0
    for m in matches:
        i=i+1
        # Generate random color for RGB/BGR and grayscale images as needed.
        if not color: 
            c = np.random.randint(0,256,3) if len(img1.shape) == 3 else np.random.randint(0,256)
            c = tuple([int(x) for x in c])        
        if matchesMask[i-1]==0: 
            continue
        end1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
        end2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int) + np.array([img1.shape[1], 0]))
        cv2.line(new_img, end1, end2, c, thickness, )
        cv2.circle(new_img, end1, r, c, thickness)
        cv2.circle(new_img, end2, r, c, thickness)
    return new_img





def _generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

def _normalize_world_keypoints(wkps, rotate=True):
    body_parts = wkps.keys()
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['HYPURAL_PLATE'] for bp in body_parts}

    if not rotate:
        return translated_wkps
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp].T) for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
    v = norm_wkps_intermediate['ADIPOSE_FIN'] - np.array([norm_wkps_intermediate['ADIPOSE_FIN'][0], 0, 0])
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp]) for bp in body_parts}
    
    # perform reflecton if necessary
    if norm_wkps['PECTORAL_FIN'][1] > 0:
        norm_wkps = {bp: np.array([
            norm_wkps[bp][0],
            -norm_wkps[bp][1],
            norm_wkps[bp][2]
        ]) for bp in body_parts}
    
    return norm_wkps


def draw_matches_3D(img1, kp1, img2, kp2, matches, matchesMask): 
    

    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    i=0
    wkps = []
    for m in matches:
        # Generate random color for RGB/BGR and grayscale images as needed.
        
        if matchesMask[i] == 1:
            p1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
            p2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int))
            p1_x_frame = p1[0] + df.left_crop_metadata.iloc[idx]['x_coord']
            p1_y_frame = p1[1] + df.left_crop_metadata.iloc[idx]['y_coord']
            p2_x_frame = p2[0] + df.right_crop_metadata.iloc[idx]['x_coord']
            params = df.camera_metadata.iloc[idx]
            disp = abs(p1_x_frame - p2_x_frame)
            depth = depth_from_disp(disp, params)
            wkp = convert_to_world_point(p1_x_frame, p1_y_frame, depth, params)
            wkps.append(wkp)
        i += 1
        
    return wkps


In [ ]:
df = bdf

In [ ]:
df.right_image_url.iloc[0]

In [ ]:
%matplotlib inline
idx = 0
left_image_f, _, _ = s3_access_utils.download_from_url(df.left_image_url.iloc[idx])
right_image_f, _, _ = s3_access_utils.download_from_url(df.right_image_url.iloc[idx])
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)

MIN_MATCH_COUNT = 10
GOOD_PERC = 0.7

sift = cv2.KAZE_create()
img1 = enhance(imageL)
img2 = enhance(imageR)
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)


flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
for m,n in matches:
    if m.distance < GOOD_PERC*n.distance:
        good.append(m)
if len(good)>=MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,7.0)
    matchesMask = mask.ravel().tolist()
else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None
    
img3 = draw_matches(img1,kp1,img2,kp2,good,matchesMask,None,False)
img3o = draw_matches(img1,kp1,img2,kp2,good,matchesMask,None,True)
alpha = 0.7  # Transparency factor.
img3 = cv2.addWeighted(img3o, alpha, img3, 1 - alpha, 0)

f, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(img3)
ax.axis("off")
plt.show()

In [ ]:
M.sum()

In [ ]:
body_wkps = draw_matches_3D(img1, kp1, img2, kp2, good, matchesMask)

In [ ]:
wkps = df.original_world_keypoints.iloc[idx]
wkps['BODY'] = np.array(body_wkps)
norm_wkps = _normalize_world_keypoints(wkps)


In [ ]:
%matplotlib notebook
body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
xs = [norm_wkps[bp][0] for bp in body_parts]
ys = [norm_wkps[bp][1] for bp in body_parts]
zs = [norm_wkps[bp][2] for bp in body_parts]
xs.extend(list(norm_wkps['BODY'][0]))
ys.extend(list(norm_wkps['BODY'][1]))
zs.extend(list(norm_wkps['BODY'][2]))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, max(xs))
ax.set_ylim3d(-0.3, 0.3)
ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

In [ ]:
df.data.apply(lambda x: x.get('breadthMms')) / 2.0

In [ ]:
bdf[(bdf.captured_at >= '2019-09-06') & (bdf.captured_at < '2019-09-08')].fish_id.unique()

<h1> Model Training </h1>

In [ ]:
def generate_train_mask(df, train_frac, randomize=True):
    x = np.zeros((df.shape[0]), dtype=bool)
    x[:int(train_frac * df.shape[0])] = True
    np.random.shuffle(x)
    mask = pd.Series(x)
    return x
    

def generate_oos_score(features_df, mask, train_size, num_eigenvectors):
    np.random.seed(0)
    columns = pairwise_distance_columns + interaction_columns_quadratic + interaction_columns_cubic

    X_train = features_df.loc[mask, columns].values
    y_train = features_df.loc[mask, 'weight'].values
    w_train = features_df.loc[mask, 'w'].values
    X_test = features_df.loc[~mask, columns].values
    y_test = features_df.loc[~mask, 'weight'].values

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_normalized = scaler.transform(X_train)

    pca = PCA(n_components=min(X_train_normalized.shape[0], X_train_normalized.shape[1]))
    pca.fit(X_train_normalized)
    explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
    idx = num_eigenvectors

    pca = PCA(n_components=idx+1)
    pca.fit(X_train_normalized)
    X_train_transformed = pca.transform(X_train_normalized)
    X_test_normalized = scaler.transform(X_test)
    X_test_transformed = pca.transform(X_test_normalized)

    reg = LinearRegression().fit(X_train_transformed, y_train, sample_weight=w_train)
    score = reg.score(X_test_transformed, y_test)

    y_pred = reg.predict(pca.transform(scaler.transform(features_df[columns].values)))
    features_df['prediction'] = y_pred
    features_df['error'] = features_df.prediction - features_df.weight
    features_df['error_pct'] = features_df.error / features_df.weight
    features_df['abs_error_pct'] = features_df.error_pct.abs()

    model = {
    'mean': scaler.mean_,
    'std': scaler.scale_,
    'PCA_components': pca.components_,
    'reg_coef': reg.coef_,
    'reg_intercept': reg.intercept_,
    'body_parts': body_parts_subset   
    }
    

    return mask, model, score



<h1> Plot one instance of train / test where the training set consists of 2000 fish </h1>

In [ ]:
# num eigenvectors = 20

np.random.seed(0)
random.seed(0)
# tdf = features_df.copy(deep=True)

# weights = []
# i = 0
# for idx, row in tdf.iterrows():
#     if i % 100 == 0:
#         print(i)
#     i += 1
#     count = tdf[tdf.gtsf_fish_identifier == row.gtsf_fish_identifier].shape[0]
#     if count > 1:
#         weights.append(1.0 / count ** 0.5)
#     else:
#         weights.append(1)
        
# tdf['w'] = weights

gtsf_fish_identifiers = list(tdf.gtsf_fish_identifier.unique())
train_size = int(0.8 * len(gtsf_fish_identifiers))
fish_ids = random.sample(gtsf_fish_identifiers, train_size)
mask = tdf.gtsf_fish_identifier.isin(fish_ids) & (tdf.captured_at <= '2019-09-01')
mask, model, score = generate_oos_score(tdf, mask, 2000, 20)



In [ ]:
%matplotlib inline
amg = AccuracyMetricsGenerator()
test_mask = (~mask) & (tdf.captured_at <= '2019-09-01')
amg.set_data(mask, tdf.prediction.values, tdf.weight.values, w=tdf.w.values)
amg.plot_predictions_vs_ground_truth(impose_bounds=True)

In [ ]:
amg = AccuracyMetricsGenerator()
amg.set_data(mask, tdf.prediction.values, tdf.weight.values, w=tdf.w.values)
amg.plot_predictions_vs_ground_truth(impose_bounds=True)

In [ ]:
tdf[tdf.error_pct > 0.5].keypoint_annotation_id

<h1> Visualize Individual Cases </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
credentials = json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS']))
rds_access_utils = RDSAccessUtils(credentials)
v = Visualizer(s3_access_utils, rds_access_utils)


In [ ]:
%matplotlib inline
keypoint_annotation_id = 724097
v.load_data(keypoint_annotation_id)
v.display_crops()

In [ ]:
%matplotlib notebook
v.display_3d_keypoints()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(features_df.rms_error_m, bins=100)
plt.grid()
plt.show()

In [ ]:
cm = plt.cm.get_cmap('seismic')
fig, ax = plt.subplots(figsize=(20, 10))
mask = features_df.rms_error_m < 0.5
sc = ax.scatter(features_df[mask].sort_values('rms_error_m').weight.values, 
            features_df[mask].sort_values('rms_error_m').prediction.values,
            c=features_df[mask].sort_values('rms_error_m').rms_error_m.values,
            cmap=cm)
plt.colorbar(sc)
plt.plot([0, 10000], [0, 10000])
# plt.xlim([0, 10000])
# plt.ylim([0, 10000])
plt.grid()
plt.show()

In [ ]:
features_df[mask].sort_values('error_pct', ascending=False).keypoint_annotation_id

In [ ]:
def centroid_depth(wkps):
    if wkps:
        mean = np.mean(np.array([wkp[1] for wkp in wkps.values()]))
        if mean == np.inf:
            return None
        return mean
    return None

# features_df['centroid_depth'] = features_df.original_world_keypoints.apply(lambda x: centroid_depth(x))
bdf['centroid_depth'] = bdf.original_world_keypoints.apply(lambda x: centroid_depth(x))

In [ ]:
def calculated_depth(wkps):
    if wkps:
        mean = np.mean([wkps['DORSAL_FIN'], wkps['']])
        if mean == np.inf:
            return None
        return mean
    return None

features_df['centroid_depth'] = features_df.original_world_keypoints.apply(lambda x: centroid_depth(x))

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.hist(features_df[(features_df.centroid_depth > 0) & (features_df.centroid_depth < 2.0)].centroid_depth, bins=100)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(features_df.centroid_depth, features_df.rms_error_m)
plt.grid()
plt.show()

In [ ]:
features_df.rms_error_m.mean()

In [ ]:
df.left_image_url.iloc[0]

In [ ]:
cm = bdf[bdf.captured_at > '2019-09-01'].camera_metadata.iloc[-1]
x_upper_lip_left = 1016
y_upper_lip_left = 1051
x_upper_lip_right = 384
x_tail_notch_left = 3926
y_tail_notch_left = 1091
x_tail_notch_right = 3360
disp_upper_lip = abs(x_upper_lip_left - x_upper_lip_right)
disp_tail_notch = abs(x_tail_notch_left - x_tail_notch_right)
depth_upper_lip = depth_from_disp(disp_upper_lip, cm)
depth_tail_notch = depth_from_disp(disp_tail_notch, cm)
wkp_upper_lip = convert_to_world_point(x_upper_lip_left, y_upper_lip_left, depth_upper_lip, cm)
wkp_tail_notch = convert_to_world_point(x_tail_notch_left, y_tail_notch_left, depth_tail_notch, cm)
print(euclidean_distance(wkp_upper_lip, wkp_tail_notch))

In [ ]:
depth_upper_lip, depth_tail_notch

In [ ]:
df[df.captured_at >= '2019-09-01'].camera_metadata

In [ ]:
bdf['gt_length'] = bdf.data.apply(lambda x: x['lengthMms'] / 1e3)
bdf['pred_length'] = bdf.original_world_keypoints.apply(lambda x: euclidean_distance(x['UPPER_LIP'], x['TAIL_NOTCH']) if x else None)
bdf['length_diff'] = bdf.pred_length - bdf.gt_length

In [ ]:
mask = (bdf.captured_at >= '2019-09-01') & (bdf.centroid_depth < 0.7)
bdf.loc[mask, ['length_diff', 'centroid_depth']]

In [ ]:
mask_1 = (bdf.captured_at <= '2019-09-10') & (bdf.centroid_depth < 0.7) & (bdf.length_diff.abs() < 0.2)
plt.figure(figsize=(20, 10))
plt.hist(bdf[mask_1].length_diff, bins=10, color='blue', alpha=0.5, density=True)
plt.grid()

mask_2 = (bdf.captured_at >= '2019-09-10') & (bdf.centroid_depth < 0.7) & (bdf.length_diff.abs() < 0.2)
plt.hist(bdf[mask_2].length_diff, bins=10, color='red', alpha=0.5, density=True)
plt.grid()
plt.show()


plt.show()

In [ ]:
mask_2 = (bdf.captured_at >= '2019-09-10') & (bdf.centroid_depth < 0.7) & (bdf.length_diff.abs() < 0.2)
plt.figure(figsize=(20, 10))
plt.hist(bdf[mask_2].length_diff, bins=20)
plt.grid()
plt.show()

In [ ]:
bdf[mask_2].sort_values('length_diff')

In [ ]:
bdf[mask_1 & (bdf.length_diff > 0.02)].sort_values('length_diff')

In [ ]:
%matplotlib inline
keypoint_annotation_id = 507364
v.load_data(keypoint_annotation_id)
v.display_crops()